# Generate plots for event-related fMRI with TDA
This notebook contains the tools for displaying the output of the event-related fMRI data analyses found at: https://github.com/CameronTEllis/event_related_fmri_tda. 

This assumes 1) you ran at least some of the simulation/analyses of the data (using "./code/supervisor_supersubject.sh" for instance). More explanation for this will be provided [below](#supersubject_script). 2) you are able to launch the jupyter notebook with the same environment that you used to run the code.

Contents:


[Setup](#setup)  

[Check that the appropriate analyses have been run](#check)  

>[Print default script for running all of the main analyses](#supersubject_script)  

[Validate the simulation accuracy](#validate)  

[Pull out the MDS representations from the fully simulated data](#mds)  

[Make max loop signal plots](#max_loop)  

[Make the plots for the matching loop number](#proportion_loop)  

[How many searchlights contain the ROI](#searchlights_in_ROI)  

[Generate the elements for the methods figure in the manuscript](#figure_1)  
>[Plot a brain slice](#figure_1_slice)  
>[Plot a distance matrix](#figure_1_dist)  
>[Run persistent homology on a searchlight voxel and plot it](#figure_1_PH)  

[Plot example signal activity and signal plus noise](#figure_S4)  

<a id="setup"></a> 
## Setup some functions that will be used throughout

In [ ]:
%matplotlib notebook

import sys
import os
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import nibabel as nb
import scipy.spatial.distance as sp_distance
import sklearn.manifold as manifold
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
from scipy.stats import t
from scipy import stats
import generate_graph_structure as graphs
import test_graph_structure as test_graphs
import glob
from IPython.core.display import HTML

In [ ]:
def average_node_brain_dist(filename, mask_vec):
    nii = nb.load(filename)
    data_vol = nii.get_data()

    # Turn the data into a vector
    data_vec = data_vol.reshape((np.prod(data_vol.shape[0:3])), data_vol.shape[3])

    # Mask the data
    data_masked = data_vec[mask_vec]

    # Average the voxels in the masked region
    data_av = np.mean(data_masked, 0)

    # Reshape the data to be node by node
    nodes = int(np.ceil(np.sqrt(len(data_av) * 2)))

    # What are the indices for the upper triangle
    idxs_u = np.triu_indices(nodes, 1)
    idxs_l = np.tril_indices(nodes, -1)

    # Insert the data into a dist matrix
    data_dist = np.zeros((nodes, nodes))

    # Add the data
    data_dist[idxs_u[0], idxs_u[1]] = data_av

    # Symmetrize the data
    data_dist = (data_dist.T + data_dist) / 2
    
    # Return the distance matrix
    return data_dist

In [ ]:
# Output a plot of the summary for a given condition
def plot_summary_stats(data, keys, line_style, color):
    
    plotting_data = []
    scatter_data = []
    plotting_error = []
    ppt_num = 20 # What is the expected number of ppts
    for key in keys:
        
        # Pull out the data
        data_point = data[key]
        
        # Is this a dictionary
        if isinstance(data_point, dict):
            # If these are different participants then average them and add error bars
            ppt_data = list(data_point.values())
            
            # Do something different for the max signal points
            if key.find('s-5.0') < 0:
                plotting_data += [np.nanmean(ppt_data)]
                plotting_error += [np.nanstd(ppt_data) / np.sqrt(len(ppt_data))]
            else:
                scatter_data += [5, np.nanmean(ppt_data)]
            
            if len(ppt_data) != ppt_num:
                print('%s has only %d ppts' % (key, len(ppt_data)))
            
        else:
            # Do something different for the max signal points
            if key.find('s-5.0') < 0:
                plotting_data += [data_point]
            else:
                scatter_data += [5, data_point]
    
    # Plot the data
    plt.plot(plotting_data, line_style, color= color, linewidth=5)
    if len(plotting_error) == len(plotting_data):
        plt.errorbar(np.arange(len(plotting_data)), plotting_data, plotting_error, linestyle=line_style, ecolor=color, color=color, linewidth=5)
    
    # Scatter the data
    if len(scatter_data) > 0:
        if line_style == '-':
            plt.plot(scatter_data[0], scatter_data[1], markeredgecolor=color, marker='o', fillstyle='none', markersize= 20, markeredgewidth=3)
    #        plt.scatter(scatter_data[0], scatter_data[1], color=color, hatch='/' * 5, alpha=0.4, s= 250)
        else:
            plt.scatter(scatter_data[0], scatter_data[1], color=color, alpha=0.5, s= 250)     
     
    # Return the data
    return plotting_data, scatter_data

In [ ]:
# Create a function to compare the different inputs
def plot_boxplot(*arg,
                 ):
    plt.figure(figsize=(4, len(arg) * 3))
    
    # Reshape and store the data

    for arg_number in list(range(0, len(arg))):
        var = arg[arg_number]
        var = var.reshape(np.prod(var.shape))

        # Add the data to the matrix
        if arg_number == 0:
            data = np.asarray([var])
        else:
            data = np.vstack([data, var])

    # What is the x value
    x_idxs = list(range(1, data.shape[0] + 1))
    
    plt.plot(x_idxs, data, c=(0.9,0.9,0.9))
    
    # What indexes are usable from the data
    useable_idxs = np.where(np.prod(~np.isnan(data), axis=0)==1)[0]
    
    # Plot the box plots
    plt.boxplot(np.transpose(data[:, useable_idxs]))
    
    # If there are two inputs, do a t test
    if len(arg) == 2:
        t, p = stats.ttest_rel(data[1, useable_idxs], data[0, useable_idxs])
        delta_mean = np.nanmean(np.subtract(data[1, useable_idxs], data[0, useable_idxs]))
        df = data.shape[1] - 1
        print('Test statistics:')
        print("M=%0.2f, t(%d)=%0.2f, p=%0.3f" % (delta_mean,df,t,p))

    
# Create all of the plots
def plot_parameter(parameter_type):

    if parameter_type == 'SNR':
        parameter_idx = 0
    elif parameter_type == 'SFNR':
        parameter_idx = 1        
    elif parameter_type == 'FWHM':
        parameter_idx = 2        
    elif parameter_type == 'AR':
        parameter_idx = 3         
    elif parameter_type == 'MA':
        parameter_idx = 4         
            
    # Make the box plot
    plot_boxplot(real_noise_dicts[:, :, parameter_idx], simulated_noise_dicts_means[:, :, parameter_idx])
    plt.title(parameter_type)
    plt.xticks(np.asarray([1,2]), ['Real', 'Simulated'])
    plt.savefig('../plots/%s_comparison.eps' % parameter_type, format='eps', dpi=100)
    ylims = plt.ylim()


def plot_persistence(barcode):

    # Pull out all the features
    births = barcode[:, 0]
    deaths = barcode[:, 1]
    betti = barcode[:, 2]

    plt.plot(births[betti==1], deaths[betti==1], marker='x', linestyle='none')
    plt.plot(births[betti==0], deaths[betti==0], marker='o', fillstyle='none', linestyle='none')

    # Get the diagonal
    all_vals = barcode[:,:2].flatten()
    min_val = all_vals.min()
    max_val = all_vals[np.argsort(all_vals)[-2]]  # So as not to get an inf

    plt.plot([min_val, max_val], [min_val, max_val], c='k')

#     # Hide units
#     plt.xticks([])
#     plt.yticks([])

#     plt.xlabel('Birth')
#     plt.ylabel('Death')
    plt.axis('equal')

In [ ]:
# Convert steps into an appropriate string for printing
def conv_steps(steps):
    step_str = str(steps)
    
    # Remove offending characeters
    step_str = step_str.replace('[', '')
    step_str = step_str.replace(']', '')
    step_str = step_str.replace(',', '')
    
    return step_str

In [ ]:
# Set plotting style
short_style = '--'
long_style = '-'
very_long_style = '-.'
low_nodes_color = np.asarray([143, 199, 54]) / 255
mid_nodes_color = np.asarray([255, 93, 54]) / 255
high_nodes_color = np.asarray([73, 94, 204]) / 255

In [ ]:
# Set the range of parameters to be searched over
signal_steps = [0.0, 0.25, 0.5, 0.75, 1.0, 5.0]
event_types = [12, 15, 18]
repetition_steps = [5, 10] 
isi = '5.0'
deconvolution = 1 # Do you want to use the deconvolution version
resample = 1 # Which resample do you want to use

In [ ]:
# Set up file names
mask_name='../simulator_parameters/real_results/significant_mask.nii.gz'
subject_root = '../simulated_data/node_brain_dist/'
supersubject_root = '../simulated_data/supersubject_node_brain_dist/'
nii = nb.load(mask_name)
mask_vol = nii.get_data()
mask_vec = mask_vol.reshape((np.prod(mask_vol.shape[0:3]))) == 1

<a id="check"></a> 
## Check that the appropriate analyses have been run
Do a quick check to ensure that the files you need for plotting here exist. This is an incomplete list but will suffice for the first few steps.

First print the commands that need to be run (make sure you set the long partition appropriately) <a id="supersubject_script"></a> 

In [ ]:
print('for signal in %s; do for events in %s; do for repetitions in %s; do sbatch -p $LONG_PARTITION ./code/supervisor_supersubject.sh elipse $signal [1,1] 1 $events $repetitions 1; done; done; done' % (conv_steps(signal_steps), conv_steps(event_types), conv_steps(repetition_steps)))


Now check whether each file exists

In [ ]:
if os.path.isfile('../validation_summary/real_noise_dicts.npy') == 0:
    print('Cannot find files for ''Validation simulation'' script. Run `./code/supervisor_generate_noise_validation.sh` and then after that is finished, run `sbatch code/run_testing_noise_calc_noise_validation.sh`\n')
    
if os.path.isfile('%s/%selipse_s-%s_1_1_t_5.0_1_1.0_%d_%d_1_resample-1.nii.gz' % (subject_root, 'sub-1_', signal_steps[0], repetition_steps[0], event_types[0])) == 0:
    print('Cannot find files for generating the MDS plots (and distance matrices and persistence diagrams). Make sure have run at least `generate_node_brain_dist.py` for the conditions. The best way to do this is to do the supersubject analysis, as printed above\n')

if os.path.isfile('../searchlight_summary/signal_vs_flipped.txt') == 0 or os.path.isfile('../searchlight_summary/signal_proportion.txt')  == 0 or os.path.isfile('../searchlight_summary/flipped_proportion.txt')  == 0 or os.path.isfile('../searchlight_summary/signal_ratio.txt')  == 0 or os.path.isfile('../searchlight_summary/flipped_ratio.txt') == 0 :
    print('Cannot find the summary of searchlight information. Make sure you have run the supervisor script, printed above, as well as the `run_test_signal_vol.sh` for each searchlight output')
    print('Consider using the following command:\nfiles=`ls simulated_data/searchlights/*`; for file in $files; do sbatch run_test_signal_vol.sh $file ./searchlight_summary/; done;\n')
    

## Validate the simulation accuracy<a id="validate"></a> 
Using an approach that was used to validate the fmrisim package we check that the noise simulation is appropriate for this data.

To run this analysis, first run `./code/supervisor_generate_noise_validation.sh` and then after that is finished, run `sbatch code/run_testing_noise_calc_noise_validation.sh`
    

In [ ]:
# Take the output of the fitting analyses and plot the heatmap of results

# Identify file. paths
testing_noise_calc_path = '../validation_summary/'

# What is the file name
real_noise_dicts = np.load(testing_noise_calc_path + 'real_noise_dicts.npy')
simulated_noise_dicts = np.load(testing_noise_calc_path + 'simulated_noise_dicts.npy')
simulated_noise_dicts_means = np.load(testing_noise_calc_path + 'simulated_noise_dicts_means.npy')
simulated_noise_dicts_std = np.load(testing_noise_calc_path + 'simulated_noise_dicts_stds.npy')


In [ ]:
plot_parameter('SNR')

In [ ]:
plot_parameter('SFNR')

In [ ]:
plot_parameter('FWHM')

In [ ]:
plot_parameter('AR')

<a id="mds"></a> 
## Pull out the MDS representations from the fully simulated data
This gives you a simple and easy way to visual the structure of the data in the signal voxels. 

This assumes that you have run `generate_node_brain_dist.py` for the relevant participants and conditions. In this case that is subject 1 and the supersubject analysis for several signal steps, event_types and repetitions per run

In [ ]:
subj = 1 # If zero then do the supersubject

for nodes in event_types:
    for repetitions_per_run in repetition_steps:

        plt.figure(figsize=(24, 8))
        for counter, signal_size in enumerate(signal_steps):

            # Set the subject name
            if subj == 0:
                subject_name = ''
                file_root = supersubject_root
            else:
                subject_name = 'sub-%d_' % subj
                file_root = subject_root

            # Create a variable with a specific dp precision
            signal_number = '%0.2f' % signal_size
            if signal_number[-1] == '0':
                signal_number = signal_number[:-1]

            filename = '%s/%selipse_s-%s_1_1_t_5.0_1_1.0_%d_%d_%d_resample-%d.nii.gz' % (file_root, subject_name, signal_number, repetitions_per_run, nodes, deconvolution, resample)
            plt.suptitle('reps-%d, nodes-%d, deconv-%d' % (repetitions_per_run, nodes, deconvolution))
            
            if os.path.exists(filename):
                data_dist=average_node_brain_dist(filename, mask_vec)
                
                plt.subplot(3, len(signal_steps), counter + 1 )
                graphs.make_mds(data_dist,
                                dim=2,
                                )
                plt.axis('equal')
                
                plt.subplot(3, len(signal_steps), counter + 1 + len(signal_steps))
                plt.imshow(data_dist)
                plt.colorbar()
                plt.axis('off')
                plt.axis('equal')
                
                # Calculate the persistent homology of this distance matrix
                barcode = test_graphs.make_persistence(data_dist)
                
                plt.subplot(3, len(signal_steps), counter + 1 + len(signal_steps) * 2)
                plot_persistence(barcode)
                
                plt.xlabel('s-%0.2f' % signal_size)
                
        plt.savefig('../plots/example_mds_%snodes_%d_repetitions_%d.eps' % (subject_name, nodes, repetitions_per_run))
        plt.savefig('../plots/example_mds_%snodes_%d_repetitions_%d.png' % (subject_name, nodes, repetitions_per_run))            


In [ ]:
subj = 0 # If zero then do the supersubject

for nodes in event_types:
    for repetitions_per_run in repetition_steps:

        plt.figure(figsize=(24, 8))
        for counter, signal_size in enumerate(signal_steps):

            # Set the subject name
            if subj == 0:
                subject_name = ''
                file_root = supersubject_root
            else:
                subject_name = 'sub-%d_' % subj
                file_root = subject_root

            # Create a variable with a specific dp precision
            signal_number = '%0.2f' % signal_size
            if signal_number[-1] == '0':
                signal_number = signal_number[:-1]

            filename = '%s/%selipse_s-%s_1_1_t_5.0_1_1.0_%d_%d_%d_resample-%d.nii.gz' % (file_root, subject_name, signal_number, repetitions_per_run, nodes, deconvolution, resample)

            duration = (repetitions_per_run * nodes * 5 * 9) / 60  # How long in minutes is this experiment
            plt.suptitle('reps-%d, nodes-%d, deconv-%d' % (repetitions_per_run, nodes, deconvolution))

            if os.path.exists(filename):
                data_dist=average_node_brain_dist(filename, mask_vec)
                
                plt.subplot(3, len(signal_steps), counter + 1 )
               
                graphs.make_mds(data_dist,
                                dim=2,
                                )
                plt.axis('equal')
                
                plt.subplot(3, len(signal_steps), counter + 1 + len(signal_steps))
                plt.imshow(data_dist)
                plt.colorbar()
                plt.axis('off')
                plt.axis('equal')
                
                # Calculate the persistence diagram
                barcode = test_graphs.make_persistence(data_dist)
                
                plt.subplot(3, len(signal_steps), counter + 1 + len(signal_steps) * 2)
                plot_persistence(barcode)
                
                plt.xlabel('s-%0.2f' % signal_size)
                
                
        plt.savefig('../plots/example_mds_%snodes_%d_repetitions_%d.eps' % (subject_name, nodes, repetitions_per_run))
        plt.savefig('../plots/example_mds_%snodes_%d_repetitions_%d.png' % (subject_name, nodes, repetitions_per_run))            


<a id="max_loop"></a> 
## Make max loop signal plots
Make the plot of the maximum loop persistence for each condition

In [ ]:
# Load in all of the summary statistics 
fid = open('../searchlight_summary/signal_vs_flipped.txt', 'r')
file_txt = fid.readlines()
fid.close()

subject_data = {}
supersubject_data = {}
for line in file_txt:
    
    # What condition is it
    condition = line[line.find('elipse'):line.find('_loop')]
    
    # What is the value
    val = float(line[line.find(': ') + 2:line.find('\n')])
    
    # Is it a supersubject or not?
    if line.find('sub-') > -1:
        
        ppt = line[line.find('sub-'):line.find('_elipse')]
        
        # Add a dictionary if it doesn't exist
        if condition not in subject_data:
            subject_data[condition] = {}
            
        # Add to the list    
        subject_data[condition][ppt] = val
        
    else:
        supersubject_data[condition] = val
        
    

In [ ]:
plt.figure()

threshold = stats.t.ppf(0.9995, 441)

# Cycle through all of the conditions
for trials in [5, 10]:
    if trials == 5:
        line_style = short_style
    elif trials == 10:
        line_style = long_style
    elif trials == 25:
        line_style = very_long_style
        
    for nodes in [12, 15, 18]:
        keys = []

        if nodes == 12:
            node_color = low_nodes_color
        elif nodes == 15:
            node_color = mid_nodes_color
        elif nodes == 18:
            node_color = high_nodes_color

        keys = []
        for s in signal_steps:

            # Store the keys
            keys += ['elipse_s-%s_1_1_t_%s_1_1.0_%s_%d_%d_resample-%d' % (str(s), isi, trials, nodes, deconvolution, resample)]

        plot_summary_stats(subject_data, keys, line_style, node_color)


plt.plot([0, len(signal_steps) - 1], [threshold, threshold], 'k--')
plt.xticks(np.arange(len(signal_steps)), signal_steps)
plt.ylabel('t stat')
plt.xlabel('Percent signal change')
plt.ylim([-10, 25])

plt.savefig('../plots/subjectwise_max_loop_tstat.eps')
plt.savefig('../plots/subjectwise_max_loop_tstat.png')            


In [ ]:
plt.figure()

# Cycle through all of the conditions
for trials in repetition_steps:
    if trials == 5:
        line_style = short_style
    elif trials == 10:
        line_style = long_style
        
    for nodes in event_types:
        keys = []

        if nodes == 12:
            node_color = low_nodes_color
        elif nodes == 15:
            node_color = mid_nodes_color
        elif nodes == 18:
            node_color = high_nodes_color

        keys = []
        for s in signal_steps:

            # Store the keys
            keys += ['elipse_s-%s_1_1_t_%s_1_1.0_%s_%d_%d_resample-%d' % (str(s), isi, trials, nodes, deconvolution, resample)]

        plot_summary_stats(supersubject_data, keys, line_style, node_color)

plt.plot([0, len(signal_steps) - 1], [threshold, threshold], 'k--')
plt.xticks(np.arange(len(signal_steps)), signal_steps)
plt.ylabel('t stat')
plt.xlabel('Percent signal change')
plt.ylim([-10, 25])

plt.savefig('../plots/supersubject_max_loop_tstat.eps')
plt.savefig('../plots/supersubject_max_loop_tstat.png')            

<a id="proportion_loop"></a> 
## Make the plots for the matching loop number
Make the plots showing the frequency of getting exactly one loop (a single point in the 1-Dimensional persistence diagram)

In [ ]:
# What method do you want to use to select the single point. 'proportion' means the number of searchlights with only one loop feature, 'ratio' means the number of loops above a threshold
threshold_type = 'proportion'  

# Load in all of the summary statistics 
fid = open('../searchlight_summary/signal_%s.txt' % threshold_type, 'r')
file_txt = fid.readlines()
fid.close()

subject_signal = {}
supersubject_signal = {}
for line in file_txt:
    
    # What condition is it
    condition = line[line.find('elipse'):line.find('_loop_counter')]
    
    # What is the value
    val = float(line[line.find(': ') + 2:line.find('\n')])
    
    # Is it a supersubject or not?
    if line.find('sub-') > -1:
        
        ppt = line[line.find('sub-'):line.find('_elipse')]
        
        # Add a dictionary if it doesn't exist
        if condition not in subject_signal:
            subject_signal[condition] = {}
            
        # Add to the list    
        subject_signal[condition][ppt] = val
    else:
        
        if condition in supersubject_signal:
            print('Condition already stored in the dict, not adding')
        else:
            supersubject_signal[condition] = val
            

In [ ]:
# What method do you want to use to select the single point. 'proportion' means the number of searchlights with only one loop feature, 'ratio' means the number of loops above a threshold
threshold_type = 'proportion'  

# Load in all of the summary statistics 
fid = open('../searchlight_summary_test/signal_%s.txt' % threshold_type, 'r')
file_txt = fid.readlines()
fid.close()

subject_signal = {}
supersubject_signal = {}
for line in file_txt:
    
    # What condition is it
    condition = line[line.find('elipse'):line.find('_loop_counter')]
    
    # What is the value
    val = float(line[line.find(': ') + 2:line.find('\n')])
    
    # Is it a supersubject or not?
    if line.find('sub-') > -1:
        
        ppt = line[line.find('sub-'):line.find('_elipse')]
        
        # Add a dictionary if it doesn't exist
        if condition not in subject_signal:
            subject_signal[condition] = {}
            
        # Add to the list    
        subject_signal[condition][ppt] = val
    else:
        
        if condition in supersubject_signal:
            print('Condition already stored in the dict, not adding')
            print(line)
        else:
            supersubject_signal[condition] = val
            

In [ ]:
# Load in all of the summary statistics 
fid = open('../searchlight_summary/flipped_%s.txt' % threshold_type, 'r')
file_txt = fid.readlines()
fid.close()

subject_noise = {}
supersubject_noise = {}
for line in file_txt:
    
    # What condition is it
    condition = line[line.find('elipse'):line.find('_loop')]
    
    # Check the general properties are correct
    if condition.find('1_1_t_5.0_1_1.0') > 0:

        # Pull out the rep number
        if condition.find('_1.0_5_1') > 0:
            rep_name = 'short'
        elif condition.find('_1.0_10_1') > 0:
            rep_name = 'long'
        else:
            rep_name = 'other_rep'   

        # Pull out the node number
        if condition.find('_12_1') > 0:
            node_name = 'low'
        elif condition.find('_15_1') > 0:
            node_name = 'mid'
        elif condition.find('_18_1') > 0:
            node_name = 'high'
        else:
            node_name = 'other_node'

        condition_name = rep_name + '_' + node_name

        # What is the value
        val = float(line[line.find(': ') + 2:line.find('\n')])

        # Is it a supersubject or not?
        if line.find('sub-') > -1:
            # Add a dictionary if it doesn't exist
            if condition_name not in subject_noise:
                subject_noise[condition_name] = []

            subject_noise[condition_name] += [val]
        else:
            # Add a dictionary if it doesn't exist
            if condition_name not in supersubject_noise:
                supersubject_noise[condition_name] = []
            supersubject_noise[condition_name] += [val]

In [ ]:
# Plot the loaded in summary statistics

plt.figure()

# Set the variables
# Cycle through all of the conditions
for trials in repetition_steps:
    if trials == 5:
        line_style = short_style
    elif trials == 10:
        line_style = long_style
        
    for nodes in event_types:
        keys = []

        if nodes == 12:
            node_color = low_nodes_color
        elif nodes == 15:
            node_color = mid_nodes_color
        elif nodes == 18:
            node_color = high_nodes_color

        keys = []
        for s in signal_steps:

            # Store the keys
            keys += ['elipse_s-%s_1_1_t_%s_1_1.0_%s_%d_%d_resample-%d' % (str(s), isi, trials, nodes, deconvolution, resample)]

        plot_summary_stats(subject_signal, keys, line_style, node_color)

# Plot the noise pattern
start_x_pos = 0.881
step_x_pos = 0.019
for node_name in [['low', low_nodes_color], ['mid', mid_nodes_color], ['high', high_nodes_color]]:
    for rep_name in [['short_', '/'  * 5], ['long_', None]]:
        
        # Plot this condition
        plt.axhspan(np.min(subject_noise[rep_name[0] + node_name[0]]), np.max(subject_noise[rep_name[0] + node_name[0]]), xmin=start_x_pos, xmax=start_x_pos + step_x_pos, alpha=0.1, color=node_name[1], hatch = rep_name[1])
        
        start_x_pos += step_x_pos + 0.001

# Plot the graph features
plt.xticks(np.arange(len(signal_steps)), signal_steps)
plt.ylabel('Prop. 1 loop')
plt.xlabel('Percent signal change')
plt.ylim([0, 1])
plt.xlim([0, 6])

plt.savefig('../plots/subjectwise_%s.eps' % threshold_type)
plt.savefig('../plots/subjectwise_%s.png' % threshold_type)            


In [ ]:
plt.figure()


# Cycle through all of the conditions
for trials in repetition_steps:
    if trials == 5:
        line_style = short_style
    elif trials == 10:
        line_style = long_style
        
    for nodes in event_types:
        keys = []

        if nodes == 12:
            node_color = low_nodes_color
        elif nodes == 15:
            node_color = mid_nodes_color
        elif nodes == 18:
            node_color = high_nodes_color

        for s in signal_steps:

            # Store the keys
            keys += ['elipse_s-%s_1_1_t_%s_1_1.0_%s_%d_%d_resample-%d' % (str(s), isi, trials, nodes, deconvolution, resample)]

        plot_summary_stats(supersubject_signal, keys, line_style, node_color)

# Plot the noise pattern
start_x_pos = 0.881
step_x_pos = 0.019
for node_name in [['low', low_nodes_color], ['mid', mid_nodes_color], ['high', high_nodes_color]]:
    for rep_name in [['short_', '/' * 5], ['long_', None]]:
        
        # Plot this condition
        plt.axhspan(np.min(supersubject_noise[rep_name[0] + node_name[0]]), np.max(supersubject_noise[rep_name[0] + node_name[0]]), xmin=start_x_pos, xmax=start_x_pos + step_x_pos, alpha=0.1, color=node_name[1], hatch = rep_name[1])
        
        start_x_pos += step_x_pos + 0.001        
        
# Plot the graph features
plt.xticks(np.arange(len(signal_steps)), signal_steps)
plt.ylabel('Prop. 1 loop')
plt.xlabel('Percent signal change')
plt.ylim([0, 1])
plt.xlim([0, 6])

plt.savefig('../plots/supersubject_%s.eps'% threshold_type)
plt.savefig('../plots/supersubject_%s.png' % threshold_type)            


<a id="ratio_loop"></a> 
## Search over different thresholds for TDA
As per the supplemental analysis, examine what is the effect of different ratio thresholds on whether one loop is identified. A value of 2 means that the persistence of the largest loop must be at least 2x the persistence of the next largest loop. 

In [ ]:
threshold_type = 'ratio'
ratio_steps = [1.1, 1.2, 1.4, 1.8, 2.6, 4.2, 7.4, 13.8]
s = '0.75'
resample = 1

Once you have run the [supervisor script](#supersubject_script) , you can run the searchlight analysis selectively for different thresholds. Use the following line:

In [ ]:
print('for signal in %s; do for events in %s; do for repetitions in %s; do for ratio in %s; do sbatch ./code/supervisor_searchlight.sh simulated_data/searchlights/ elipse_s-${signal}_1_1_t_5.0_1_1.0_${events}_${repetitions}_1_resample-1 loop_ratio_${ratio}; done; done; done' % (conv_steps(s), conv_steps(event_types), conv_steps(repetition_steps), conv_steps(ratio_steps)))

Once that is done, run the following line to summarise these volumes:

In [ ]:
print('files=`ls simulated_data/searchlights/*loop_ratio_*`; for file in $files; do sbatch run_test_signal_vol.sh $file ./searchlight_summary/; done;\n')


In [ ]:
# Load in all of the summary statistics 
fid = open('../searchlight_summary/signal_%s.txt' % threshold_type, 'r')
file_txt = fid.readlines()
fid.close()

subject_signal = {}
supersubject_signal = {}
for ratio_threshold in ratio_steps:

    for line in file_txt:

        # What condition is it
        start_idx = line.find('elipse')
        end_idx = line.find('_loop_ratio_%0.1f' % ratio_threshold)
        
        # Ensure that it is the correct condition
        if start_idx > -1 and end_idx > -1:
            
            condition = line[start_idx:end_idx]

            condition += '_loop_ratio_%0.1f' % ratio_threshold

            # What is the value
            val = float(line[line.find(': ') + 2:line.find('\n')])

            # Is it a supersubject or not?
            if line.find('sub-') > -1:

                ppt = line[line.find('sub-'):line.find('_elipse')]

                # Add a dictionary if it doesn't exist
                if condition not in subject_signal:
                    subject_signal[condition] = {}

                # Add to the list    
                subject_signal[condition][ppt] = val

            else:
                supersubject_signal[condition] = val 
   

fid = open('../searchlight_summary/flipped_%s.txt' % threshold_type, 'r')
file_txt = fid.readlines()
fid.close()

subject_noise = {}
supersubject_noise = {}
for ratio_threshold in ratio_steps:

    for line in file_txt:

        # What condition is it
        start_idx = line.find('elipse')
        end_idx = line.find('_loop_ratio_%0.1f' % ratio_threshold)
        
        # Ensure that it is the correct condition
        if start_idx > -1 and end_idx > -1:
        
            condition = line[start_idx:end_idx]

            condition += '_loop_ratio_%0.1f' % ratio_threshold

            # What is the value
            val = float(line[line.find(': ') + 2:line.find('\n')])

            # Is it a supersubject or not?
            if line.find('sub-') > -1:

                ppt = line[line.find('sub-'):line.find('_elipse')]

                # Add a dictionary if it doesn't exist
                if condition not in subject_noise:
                    subject_noise[condition] = {}

                # Add to the list    
                subject_noise[condition][ppt] = val
            else:
                supersubject_noise[condition] = val
            

In [ ]:
# Cycle through all of the conditions

for trials in repetition_steps:

    plt.figure(figsize=(10,5))
    if trials == 5:
        line_style = short_style
    elif trials == 10:
        line_style = long_style
        
    for nodes in event_types:
        keys = []

        if nodes == 12:
            node_color = low_nodes_color
        elif nodes == 15:
            node_color = mid_nodes_color
        elif nodes == 18:
            node_color = high_nodes_color

        for ratio in ratio_steps:

            # Store the keys
            keys += ['elipse_s-%s_1_1_t_%s_1_1.0_%s_%d_%d_resample-%d_loop_ratio_%0.1f' % (s, isi, trials, nodes, deconvolution, resample, ratio)]
        

        sig_data = plot_summary_stats(subject_signal, keys, long_style, node_color)[0]
        noise_data = plot_summary_stats(subject_noise, keys, ':', node_color)[0]
        
        # Plot the difference between signal and noise
        plt.plot(np.asarray(sig_data) - np.asarray(noise_data), c=node_color, linewidth=5)

    # Plot for this set of conditions
    plt.xticks(np.arange(len(ratio_steps)), ratio_steps)  
    plt.ylabel('Prop. 1 loop')
    plt.ylim([0, 1])
    plt.xlabel('Ratio threshold') 
    
    # Save before adding some elements you don't want in the final fig
    plt.savefig('../plots/subject_%s_repetitons-%d.eps'% (threshold_type, trials))
    plt.savefig('../plots/subject_%s_repetitons-%d.png'% (threshold_type, trials))
    
    plt.title('PSC-%s_trials-%s_nodes-%d' % (s, trials, nodes))  
    plt.legend(['Signal', 'Noise', 'Diff'] * 3)
    

<a id="searchlights_in_ROI"></a> 
## How many searchlights contain the ROI
Run a (fast) searchlight in which you count how many voxels of the signal ROI are in the searchlight. This can tell you how many searchlights you run will contain signal voxels

In [ ]:
# Import modules
import numpy as np
import nibabel
from brainiak.searchlight.searchlight import Searchlight
import os
import sys
from test_graph_structure import sl_kernel
from mpi4py import MPI

# Load in the signal mask
mask_file = '/gpfs/milgram/project/turk-browne/users/ce325/event_related_fmri_tda/simulator_parameters/real_results/significant_mask.nii.gz'

def sl_kernel(data, mask, myrad, bcvar):
    # How many mask voxels are there
    return data[0].sum()

# Load in the signal_mask
nii = nibabel.load(mask_file)
dimsize=nii.header.get_zooms()
signal_mask = nii.get_data()

# Look over all voxels in the brain (slow but easy)
mask = np.ones(signal_mask.shape)

signal_mask = signal_mask.reshape((signal_mask.shape[0], signal_mask.shape[1], signal_mask.shape[2], 1))

# Create searchlight object
sl = Searchlight(sl_rad=1, max_blk_edge=5)

# Distribute data to processes
sl.distribute([signal_mask], mask)
sl.broadcast(None)

# Run clusters
sl_outputs = sl.run_searchlight(sl_kernel)

sl_outputs = sl_outputs.astype('double')
sl_outputs[np.isnan(sl_outputs)] = 0

print('Searchlights with at least one voxel in the signal ROI', (sl_outputs >= 1).sum())
print('Searchlights with at least ten voxel in the signal ROI', (sl_outputs >= 10).sum())
print('Searchlights with all 27 voxel in the signal ROI', (sl_outputs == 27).sum())


<a id="figure_1"></a> 
## Generate the elements for the methods figure in the manuscript
This figure shows the topological structure being inserted in to the brain, the ROI containing signal, an example distance matrix and an example persistence diagram

In [ ]:
coords=graphs.elipse(nodes=12, x_coef=1, y_coef=1)
dist=graphs.coord2dist(coords)
plt.figure()
graphs.make_mds(dist,
                dim=2,
                )
plt.axis('off');
plt.savefig('../plots/example_loop.eps')
plt.savefig('../plots/example_loop.png')

<a id="figure_1_slice"></a> 
### Plot a brain slice
Take a slice of the participant data and plot it. 

To make this participant you need to output the raw simulated data, which is not stored when you run with resampling on. Hence you need to run the following line:

`sbatch -p $LONG_PARTITION ./code/supervisor_supersubject.sh elipse 5.0 [1,1] 0 5 12 1`


In [ ]:
participant_num = '1'
participant_condition = 'elipse_s-5.0_1_1_t_5.0_1_1.0_5_12_1'

In [ ]:
filename = '../simulated_data/nifti/sub-%s_r1_%s.nii.gz' % (participant_num, participant_condition)

nii = nb.load(filename)
vol = nii.get_data()

# Pull out the slices to show
x_idx = 11
brain_slice = np.rot90(vol[x_idx, :, :, 0], 1)
mask_slice = np.rot90(mask_vol[x_idx, :, :], 1)

# Find an example searchlight
y_idx = np.where(mask_slice == 1)[0][10]
z_idx = np.where(mask_slice == 1)[1][10]

# Create a searchlight mask
searchlight_slice = np.zeros(mask_slice.shape)
searchlight_slice[y_idx - 1:y_idx + 2, z_idx - 1:z_idx + 2] = 1

# Set the range to 1
brain_slice /= brain_slice.max()
mask_slice /= mask_slice.max()

# Construct RGB version of grey-level image
img_rgb = np.dstack((brain_slice, brain_slice, brain_slice))

mask_rgb = np.zeros(img_rgb.shape)
mask_rgb[:, :, 2] = mask_slice

searchlight_rgb = np.zeros(img_rgb.shape)
searchlight_rgb[:, :, 1] = searchlight_slice

# Convert the input image and color mask to Hue Saturation Value (HSV)
# colorspace
img_hsv = colors.rgb_to_hsv(img_rgb)
mask_hsv = colors.rgb_to_hsv(mask_rgb)
searchlight_hsv = colors.rgb_to_hsv(searchlight_rgb)

# Replace the hue and saturation of the original image
# with that of the color mask
alpha = 0.7
img_hsv[:, :, 0] = mask_hsv[:, :, 0]
img_hsv[:, :, 1] = mask_hsv[:, :, 1] * alpha

# Super impose the searchlight
roi_hue = np.unique(searchlight_hsv[:, :, 0])[1]
roi_sat = np.unique(searchlight_hsv[:, :, 1])[1]
idxs = np.where(searchlight_hsv[:, :, 0] == roi_hue)

img_hsv[idxs[0], idxs[1], 0] = roi_hue
img_hsv[idxs[0], idxs[1], 1] = roi_sat

# Convert back into rgb
img_masked = colors.hsv_to_rgb(img_hsv)

plt.figure()
plt.imshow(img_masked)
plt.axis('off')

# Example brain
plt.savefig('../plots/example_brain_searchlight.eps')
plt.savefig('../plots/example_brain_searchlight.png')

<a id="figure_1_dist"></a> 
### Plot a distrance matrix
Load in the distance matrix of a participant, average all of the voxels in the signal ROI and then plot the outcine

In [ ]:
filename = '../simulated_data/node_brain_dist//sub-%s_%s.nii.gz' % (participant_num, participant_condition)

data_dist = average_node_brain_dist(filename, mask_vec)

plt.figure()
plt.imshow(data_dist)
plt.axis('off')
plt.savefig('../plots/example_distance_mat.eps')
plt.savefig('../plots/example_distance_mat.png')

<a id="figure_1_PH"></a> 
### Run persistent homology on a searchlight voxel and plot it

Get the persistence diagram from a specific ROI that is specified by the searchlight above

In case the module for TDA is not easily loaded into your notebooks, use the script as specified below to run persistent homology outside of the notebook and then load in the output

In [ ]:
# Create an example barcode
input_file = 'simulated_data/node_brain_dist//sub-%s_%s.nii.gz' % (participant_num, participant_condition) # Use path relative to the base
output_file = 'simulated_data/example_barcodes/sub-%s_%s_coord.npy' % (participant_num, participant_condition) # Use path relative to the base
coordinates = '[%d,%d,%d]' % (x_idx,y_idx,z_idx)


In [ ]:
%%bash -s "$input_file" "$coordinates" "$output_file"
cd ../; ./code/run_TDA_coordinate.sh $1 $2 $3; cd code/ 

In [ ]:
# Load the barcode
barcode = np.load('../' + output_file)

In [ ]:

plt.figure()
plot_persistence(barcode)
plt.savefig('../plots/example_persistence_diagram.png')
plt.savefig('../plots/example_persistence_diagram.eps')

<a id="figure_S4"></a> 
## Plot example signal activity and signal plus noise
Run a version of the pipeline using only one run and storing useful outputs for plotting. Then use this data to make examples of the signal and noise as it is transformed between steps

In [ ]:
# Determine the signal parameters
participant_counter = 1  # Determines what noise you use
signal_structure = 'elipse' # What is the shape to be inserted (elipse is not necessarily an elipse, depends on the signal properties)
signal_properties = [1, 1]  # In the case of an elipse, what is the width and height
signal_magnitude = signal_steps[-2] # What is the PSC for each voxel
num_nodes = event_types[0] # How many nodes are you simulating
repetitions_per_run = repetition_steps[0] # How many events per run are you simulating?


In [ ]:
# Import modules
import numpy as np
import nibabel
import matplotlib.pyplot as plt
import generate_graph_structure as graphs
import copy
import sys
from scipy.stats import zscore
from sklearn import linear_model
from brainiak.utils import fmrisim as sim
import logging
import os
from nilearn.image import smooth_img
np.seterr(divide='ignore')  # Not relevant for this script

logger = logging.getLogger(__name__)

# Concatenate the timing information needed
timing_properties = [5.0,1,1.0,repetitions_per_run,num_nodes,1]
resample = 1

# If it is not a resample, do you want to save some plots
save_signal_func = 0  # Save the plot of the signal function?
save_functional = 1  # Do you want to save the functional (with node_brain)

# Inputs for generate_signal
temporal_res = 100  # How many samples per second are there for timing files
tr_duration = 2  # Force this value
fwhm = 0 # What is the fwhm of the smoothing kernel
all_pos = 1  # Assume all activation is positive

# Set analysis parameters
hrf_lag = 2 # How many TRs is the event offset by?
zscore_time = 1  # Do you want to z score each voxel in time
zscore_volume = 0  # Do you want to z score a volume

# What are the timing properties
minimum_isi = timing_properties[0]
randomise_timing = timing_properties[1]
event_durations = [timing_properties[2]]  # Assume the events are 1s long
repetitions_per_run = timing_properties[3]  # How many nodes are you simulating
nodes = timing_properties[4]  # How many nodes are to be created
deconvolution = timing_properties[5] # Do you want to use the deconvolution method for aggregating across events


## Set definitions

def regress_voxels(voxel_time,
                   design,
                   intercept=0,
                   ):
    # Takes in a voxel by time matrix and a TR by condition design matrix.
    # Assume that the timecourse used to create the design matrix was mean
    # centred
    
    # Calculate the coefficients for each condition
    voxels_reg = np.zeros((voxel_time.shape[0], design.shape[1] + intercept))
    voxels_err = np.zeros((voxel_time.shape[0], 1))
    voxels_r2 = np.zeros((voxel_time.shape[0], 1))
    ols = linear_model.LinearRegression()
    for voxel_counter in list(range(0, voxel_time.shape[0])):

        # Get this voxel
        voxel = voxel_time[voxel_counter, :]

        # If this is more than a single column design matrix go
        if design.shape[1] > 1:
            if np.all(np.isnan(voxel)) == 0:

                # What are the coefficients of each feature in the regression
                fit = ols.fit(design, voxel)
                coefs = fit.coef_

                # Calculate an R squared between the predicted response (based
                # on the coefficients) and the observed response
                predicted = np.mean((design * coefs) + fit.intercept_, 1)
                r2 = np.corrcoef(voxel, predicted)[0, 1] ** 2

                # Do you want to also add the intercept to the output?
                if intercept == 1:
                    voxels_reg[voxel_counter, :] = np.append(coefs, fit.intercept_)
                else:
                    voxels_reg[voxel_counter, :] = coefs

                voxels_err[voxel_counter, 0] = np.sqrt(np.sum((voxel - predicted) ** 2))
                voxels_r2[voxel_counter, 0] = r2

            else:
                voxels_reg[voxel_counter, :] = np.nan
                voxels_err[voxel_counter, 0] = np.nan
                voxels_r2[voxel_counter, 0] = np.nan
        else:

            # Correlate the voxels and the design
            voxels_r2[voxel_counter, 0] = np.corrcoef(voxel, design[:,
                                                             0])[0, 1]

    # # Convert all of the nans
    # voxels_reg[np.isnan(voxels_reg)] = 0

    # Output
    if design.shape[1] > 1:
        return voxels_reg, voxels_r2, voxels_err
    else:
        return voxels_r2


signal_properties_str = str(signal_properties)
signal_properties_str = signal_properties_str.replace('[', '_').replace(',',
                                                                      '_').replace(']', '').replace(' ', '')

timing_properties_str = str(timing_properties)
timing_properties_str = timing_properties_str.replace('[', '_').replace(',',
                                                                      '_').replace(']', '').replace(' ', '')

# What is the participant name with these parameters (change from default)
effect_name = '_' + signal_structure
effect_name = effect_name + '_s-' + str(signal_magnitude)
effect_name = effect_name + signal_properties_str
effect_name = effect_name + '_t' + timing_properties_str

# Only add this at the end
if resample > 0:
    effect_name = effect_name + '_resample-' + str(resample)

# Specify the paths and names
parameters_path = '../simulator_parameters/'
simulated_data_path = '../simulated_data/'
timing_path = parameters_path + 'timing/'

participant = 'sub-' + str(participant_counter + 1)
SigMask = parameters_path + '/real_results/significant_mask.nii.gz'
savename = participant + effect_name
node_brain_save = simulated_data_path + '/node_brain/' + savename + \
                  '.nii.gz'

# Load data
print('Loading ' + participant)

# Load significant voxels
nii = nibabel.load(SigMask)
signal_mask = nii.get_data()

dimsize = nii.header.get_zooms()  # x, y, z and TR size

if signal_structure == 'community_structure':

    # Load the timing information
    nodes = 15  # overwrite
    onsets_runs = np.load(timing_path + participant + '.npy')

else:
    # Generate the timing for this participant
    base_runs = np.load(timing_path + participant + '.npy')
    runs = len(base_runs)
    onsets_runs = [-1] * runs
    for run_counter in list(range(runs)):
        
        # If the number of repetitions per run is zero then use Schapiro's data to calculate it
        if repetitions_per_run == 0:
            
            # How many events in this run for this participant
            events = sum(len(event) for event in base_runs)

            # How many events per node for this run?
            repetitions_per_run = int(events / nodes)
        
        current_time = 0  # Initialize
        onsets = [-1] * nodes  # Ignore first entry
        for hamilton_counter in list(range(0, repetitions_per_run)):

            # Pick on this hamilton the starting node and the direction
            node = np.random.randint(0, nodes - 1)
            direction = np.random.choice([-1, 1], 1)[0]

            # Loop through the events
            for node_counter in list(range(0, nodes)):

                # Append this time
                if np.all(onsets[node] == -1):
                    onsets[node] = np.array(current_time)
                else:
                    onsets[node] = np.append(onsets[node],
                                             current_time)

                # What is the isi?
                isi = minimum_isi + (np.random.randint(3) * tr_duration)

                # Increment the time
                current_time += event_durations[0] + isi

                # Update the node
                node += direction
                if node >= nodes:
                    node = 0
                elif node < 0:
                    node = (nodes - 1)

        # Store the runs
        print('There are %d onsets in run %d' % (len(onsets), run_counter))
        onsets_runs[run_counter] = onsets

# How many runs are there?
runs = 1

# Generate the indexes of all voxels that will contain signal
vector_size = int(signal_mask.sum())

# Find all the indices that are significant
idx_list = np.where(signal_mask == 1)

idxs = np.zeros([vector_size, 3])
for idx_counter in list(range(0, len(idx_list[0]))):
    idxs[idx_counter, 0] = idx_list[0][idx_counter]
    idxs[idx_counter, 1] = idx_list[1][idx_counter]
    idxs[idx_counter, 2] = idx_list[2][idx_counter]

idxs = idxs.astype('int8')    
    
# What voxels are they
dimensions = signal_mask.shape

# Cycle through the runs and generate the data
node_brain = np.zeros([dimensions[0], dimensions[1], dimensions[2],
                       nodes], dtype='double')  # Preset

# Generate the graph structure (based on the ratio)
if signal_structure == 'community_structure':
    signal_coords = graphs.community_structure(signal_properties[0],
                                              )
elif signal_structure == 'parallel_rings':
    signal_coords = graphs.parallel_rings(nodes,
                                          signal_properties[0],
                                          signal_properties[1],
                                         )
elif signal_structure == 'figure_eight':
    signal_coords = graphs.figure_eight(nodes,
                                        signal_properties[0],
                                        signal_properties[1],
                                       )
elif signal_structure == 'chain_rings':
    signal_coords = graphs.chain_rings(nodes,
                                       signal_properties[0],
                                       signal_properties[1],
                                      )
elif signal_structure == 'dispersed_clusters':
    signal_coords = graphs.dispersed_clusters(nodes,
                                              signal_properties[0],
                                              signal_properties[1],
                                              signal_properties[2],
                                             )
elif signal_structure == 'tendrils':
    signal_coords = graphs.tendrils(nodes,
                                    signal_properties[0],
                                    signal_properties[1],
                                   )
elif signal_structure == 'elipse':
    signal_coords = graphs.elipse(nodes,
                                  signal_properties[0],
                                  signal_properties[1],
                                 )


# Perform an orthonormal transformation of the data
if vector_size > signal_coords.shape[1]:
    signal_coords = graphs.orthonormal_transform(vector_size,
                                                      signal_coords)

# Make a backup for later    
signal_coords_ortho = np.copy(signal_coords)    
    
# Do you want these coordinates to be all positive? This means that
# these coordinates are represented as different magnitudes of
# activation
if all_pos == 1:
    mins = np.abs(np.min(signal_coords, 0))
    for voxel_counter in list(range(0, len(mins))):
        signal_coords[:, voxel_counter] += mins[voxel_counter]

# Bound the value to have a max of 1 so that the signal magnitude is more interpretable
signal_coords /= np.max(signal_coords)

# Make a backup for later    
signal_coords_all_pos = np.copy(signal_coords)    

# Get run specific names
template_name = parameters_path + 'template/' + participant + '_r' + \
                str(run_counter) + '.nii.gz'
noise_dict_name = parameters_path + 'noise_dict/' + participant + '_r' + str(run_counter) + \
                  '.txt'
nifti_save = simulated_data_path + 'nifti/' + participant + '_r' + str(run_counter)\
             + effect_name + '.nii.gz'
signal_func_save = './plots/' + participant + '_r' +\
                   str(run_counter) + effect_name + '.png'

# Load the template (not yet scaled
nii = nibabel.load(template_name)
template = nii.get_data()  # Takes a while

# Create the mask and rescale the template
mask, template = sim.mask_brain(template,
                                mask_self=True,
                                )

# Make sure all the signal voxels are within the mask
signal_mask_run = signal_mask * mask

# Pull out the onsets for this participant (copy it so you don't alter it)
onsets = copy.deepcopy(onsets_runs[run_counter - 1])

# Do you want to randomise the onsets (so that the events do not have a
# fixed order)
if randomise_timing == 1:

    # Extract all the timing information
    onsets_all = onsets[0]
    for node_counter in list(range(1, nodes)):
        onsets_all = np.append(onsets_all, onsets[node_counter])

    print('Randomizing timing across nodes')

    # Shuffle the onsets
    np.random.shuffle(onsets_all)

    # Insert the shuffled onsets back in
    for node_counter in list(range(0, nodes)):
        node_num = len(onsets[node_counter])
        onsets[node_counter] = np.sort(onsets_all[0:node_num])
        onsets_all = onsets_all[node_num:] # Remove the onsets

# If you want to use different timing then take the order of the data
# and then create a new timecourse
if minimum_isi > 1:

    # Iterate through all the onset values
    onsets_all = []  # Reset
    for node_counter in list(range(0, nodes)):
        onsets_all = np.append(onsets_all, onsets[node_counter])

        # Take the idx of all of the elements in onset all
        Idxs = np.zeros((len(onsets[node_counter]), 2))

        Idxs[:, 0] = [node_counter] * len(onsets[node_counter])
        Idxs[:, 1] = list(range(0, len(onsets[node_counter])))

        # Append the indexes
        if node_counter == 0:
            onset_idxs = Idxs
        else:
            onset_idxs = np.concatenate((onset_idxs, Idxs))

    # Change the values of the onsets
    sorted_idxs = np.ndarray.argsort(onsets_all)
    cumulative_add = 0
    for onset_counter in list(range(0, len(onsets_all))):
        # What is the onset being considered
        onset = onsets_all[sorted_idxs[onset_counter]]

        # Add time to this onset
        onsets_all[sorted_idxs[onset_counter]] = onset + cumulative_add

        # Insert the onsets at the right time
        node_counter = int(onset_idxs[sorted_idxs[onset_counter], 0])
        idx_counter = int(onset_idxs[sorted_idxs[onset_counter], 1])
        onsets[node_counter][idx_counter] = onset + cumulative_add

        cumulative_add += minimum_isi - 1

# How long should you model
last_event = max(map(lambda x: x[-1], onsets))  # Find the max of maxs
duration = int(last_event + 10) # Add a decay buffer

# Specify the dimensions of the volume to be created
dimensions = np.array([template.shape[0], template.shape[1],
                       template.shape[2], int(duration / tr_duration)])

# Load the noise parameters in
with open(noise_dict_name, 'r') as f:
    noise_dict = f.read()

noise_dict = eval(noise_dict)

# Preset brain size
brain_signal = np.zeros([dimensions[0], dimensions[1], dimensions[2],
                         int(duration / tr_duration)], dtype='double')
for node_counter in list(range(0, nodes)):

    print('Node ' + str(node_counter))

    # Preset value
    volume = np.zeros(dimensions[0:3])

    # Pull out the signal template
    signal_pattern = signal_coords[node_counter, :]
    onsets_node = onsets[node_counter]

    # Create the time course for the signal to be generated
    stimfunc = sim.generate_stimfunction(onsets=onsets_node,
                                         event_durations=event_durations,
                                         total_time=duration,
                                         temporal_resolution=temporal_res,
                                         )

    # Aggregate the timecourse
    if node_counter == 0:
        stimfunc_all = np.zeros((len(stimfunc), vector_size))
        for voxel_counter in list(range(0, vector_size)):
            stimfunc_all[:, voxel_counter] = np.asarray(
                stimfunc).transpose() * signal_pattern[voxel_counter]
    else:

        # Add these elements together
        temp = np.zeros((len(stimfunc), vector_size))
        for voxel_counter in list(range(0, vector_size)):
            temp[:, voxel_counter] = np.asarray(stimfunc).transpose() * signal_pattern[voxel_counter]

        stimfunc_all += temp

# After you have gone through all the nodes, convolve the HRF and
# stimulation for each voxel
print('Convolving HRF')
signal_func = sim.convolve_hrf(stimfunction=stimfunc_all,
                               tr_duration=tr_duration,
                               temporal_resolution=temporal_res,
                               )
if save_signal_func == 1 and resample == 0 and run_counter == 1:
    plt.plot(stimfunc_all[::int(temporal_res * tr_duration), 0])
    plt.plot(signal_func[:,0])
    plt.xlim((0, 200))
    plt.ylim((-1, 5))
    plt.savefig(signal_func_save)

# Convert the stim func into a binary vector of dim 1
stimfunc_binary = np.mean(np.abs(stimfunc_all)>0, 1) > 0
stimfunc_binary = stimfunc_binary[::int(tr_duration * temporal_res)]

# Bound, can happen if the duration is not rounded to a TR
stimfunc_binary = stimfunc_binary[0:signal_func.shape[0]]

# Create the noise volumes (using the default parameters)
noise = sim.generate_noise(dimensions=dimensions[0:3],
                           stimfunction_tr=stimfunc_binary,
                           tr_duration=tr_duration,
                           template=template,
                           mask=mask,
                           noise_dict=noise_dict,
                           )

# Change the type of noise
noise = noise.astype('double')

# Create a noise function (same voxels for signal function as for noise)
noise_function = noise[idxs[:, 0], idxs[:, 1], idxs[:, 2], :].T

# Compute the signal magnitude for the data
signal_func_scaled = sim.compute_signal_change(signal_function=signal_func,
                                               noise_function=noise_function,
                                               noise_dict=noise_dict,
                                               magnitude=[
                                                   signal_magnitude],
                                               method='PSC',
                                               )

# Multiply the voxels with signal by the HRF function
brain_signal = sim.apply_signal(signal_function=signal_func_scaled,
                                volume_signal=signal_mask,
                                )

# Convert any nans to 0s
brain_signal[np.isnan(brain_signal)] = 0

# Combine the signal and the noise (as long as the signal magnitude is above 0)
if signal_magnitude >= 0:

    # Combine the signal and the noise
    brain = brain_signal + noise

else:
    # Don't make signal just add it here
    brain = brain_signal

# Do you want to perform smoothing
if fwhm > 0:

    # Convert to nifti
    brain_nifti = nibabel.Nifti1Image(brain, nii.affine)

    # Run the smoothing step

    sm_nii = smooth_img(brain_nifti, fwhm)

    # Pull out the data again and continue on
    brain = sm_nii.get_data()

# Z score the data
brain_Z = zscore(brain, 3)

# Mask brain
brain_signal = brain_signal * mask.reshape((mask.shape[0], mask.shape[1], mask.shape[
    2], 1))

brain_Z = brain_Z * mask.reshape((mask.shape[0], mask.shape[1], mask.shape[
    2], 1))


# Find the representation of each node. You can use a deconvolution approach in which I create a design matrix, convolve it with an HRF and then regress each voxel against this matrix, storing the beta values. Alternatively you can just average the TRs N seconds after event onset
if deconvolution == 1:

    # Preset
    design_mat = np.zeros((brain.shape[3], nodes))

    for node in list(range(0, nodes)):

        # Create a stim function for just this node
        stimfunc = sim.generate_stimfunction(onsets=onsets[node],
                                 event_durations=event_durations,
                                 total_time=duration,
                                 temporal_resolution=temporal_res,
                                 )

        # Convolve it with the HRF
        signal_func = sim.convolve_hrf(stimfunction=stimfunc,
                                       tr_duration=tr_duration,
                                       temporal_resolution=temporal_res,
                                       )

        # Store in the design matrix
        design_mat[:, node] = signal_func.T

    # Make a voxel by time array
    voxel_time = brain_signal.reshape(np.prod(brain_signal.shape[0:3]), brain_signal.shape[3])

    # Regress each voxel in the brain with the design matrix
    voxels_reg, _, _ = regress_voxels(voxel_time,
                                      design_mat,
                                     )

    # Unravel the data in voxels_reg
    signal_brain = voxels_reg.reshape(brain.shape[0], 
                                        brain.shape[1],
                                        brain.shape[2],
                                        nodes)


    # Make a voxel by time array
    voxel_time = brain_Z.reshape(np.prod(brain.shape[0:3]), brain.shape[3])

    # Regress each voxel in the brain with the design matrix
    voxels_reg, _, _ = regress_voxels(voxel_time,
                                      design_mat,
                                     )

    # Unravel the data in voxels_reg
    node_brain = voxels_reg.reshape(brain.shape[0], 
                                        brain.shape[1],
                                        brain.shape[2],
                                        nodes)

    

In [ ]:
# Plot all of the relevant data from this simulated run

vox = 101 # Take the specified voxel index

# Plot the simulated response to each event for a given signal voxel
plt.figure(figsize=(6,2))
plt.bar(np.arange(num_nodes), signal_coords_all_pos[:, vox], color='k')
plt.title('Example voxel coordinate for each event type')
plt.xticks(np.arange(0, num_nodes, 2) + 1, np.arange(0, num_nodes, 2) + 2)
plt.ylabel('Weight')
plt.xlabel('Event')
plt.savefig('../plots/methods_voxel_weights.eps')

# Plot the simulated response of each voxel before any HRF convolution
plt.figure(figsize=(6,2))
plt.plot(stimfunc_all[:, vox], color='k')
plt.xticks(np.arange(0, stimfunc_all.shape[0], temporal_res * 200), np.arange(0, stimfunc_all.shape[0] / temporal_res, 200) / 2)
plt.title('Hypothesized voxel response to each experimental event, before convolution')
plt.xlabel('TRs')
plt.ylabel('Weight')
plt.savefig('../plots/methods_stimfunc.eps')

# Plot the simulated response of each voxel after convolution and scaling to the PSC
plt.figure(figsize=(6,2))
plt.plot(signal_func_scaled[:, vox])
plt.title('Hypothesized voxel response (after convolution and scaling to 1% PSC)')
plt.ylabel('PSC')
plt.xlabel('TRs')
plt.savefig('../plots/methods_convolved.eps')

# Plot the simulated noise
plt.figure(figsize=(6,2))
plt.title('Noise timecourse for voxel')
plt.plot(noise[signal_mask == 1][vox, :], color='y')
plt.xlabel('TRs')
plt.ylabel('MR value')
plt.savefig('../plots/methods_noise.eps')

# Plot the weights for each event without noise
plt.figure(figsize=(6,2))
plt.title('Deconvolved weights for signal without noise')
plt.bar(np.arange(num_nodes), signal_brain[signal_mask == 1][vox, :], color='k')
plt.xticks(np.arange(0, num_nodes, 2) + 1, np.arange(0, num_nodes, 2) + 2)
plt.ylabel('Beta weights')
plt.xlabel('Event')
plt.savefig('../plots/methods_signal_weights.eps')

# Plot the weights for each event with noise
plt.figure(figsize=(6,2))
plt.title('Deconvolved weights for signal plus noise (after z-scoring)')
plt.bar(np.arange(num_nodes), node_brain[signal_mask == 1][vox, :], color='g')
plt.xticks(np.arange(0, num_nodes, 2) + 1, np.arange(0, num_nodes, 2) + 2)
plt.ylabel('Beta weights')
plt.xlabel('Event')
plt.savefig('../plots/methods_brain_weights.eps')
